# 🎛️ Inférence Complète : Matcha-TTS + HiFi-GAN

Ce notebook combine la logique de vos fichiers `generate.py` et `generate_HifiGan.py`.
Il permet de :
1. Charger le dernier checkpoint de Matcha-TTS.
2. Charger le vocoder HiFi-GAN.
3. Effectuer la génération **manuellement** (Solver ODE Euler) comme dans vos scripts.
4. Dénormaliser le spectrogramme (si nécessaire).
5. Générer l'audio et l'écouter directement.

In [26]:
# 1. Imports & Configuration des Chemins
import sys
import os
import glob
import json
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# Ajout des chemins locaux comme dans vos scripts
sys.path.append(os.getcwd())       # Racine du projet
sys.path.append('./hifi_gan')      # Dossier HiFi-GAN
sys.path.append('./matcha')        # Dossier Matcha

# Imports spécifiques au projet
from matcha.models.matcha_tts import MatchaTTS
from matcha.text_to_ID.text_to_sequence import text_to_sequence

# Imports HiFi-GAN (avec gestion d'erreur si le dossier manque)
try:
    from hifi_gan.models import Generator as HiFiGAN
    from hifi_gan.env import AttrDict
    HIFIGAN_AVAILABLE = True
except ImportError:
    print("⚠️ Module HiFi-GAN non trouvé. Assurez-vous d'avoir le dossier 'hifi_gan' ou 'matcha/hifigan_wrapper.py'.")
    HIFIGAN_AVAILABLE = False

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device utilisé : {DEVICE}")

Device utilisé : cuda


In [27]:
# 2. Configuration

# Texte à synthétiser
TEXTE = "Hello, this is a test to verify that the notebook works exactly like your scripts."

# Paramètres de génération
N_STEPS = 10        # Nombre d'étapes pour le Solver ODE (10-50)
TEMPERATURE = 0.667 # Variance du bruit initial
LENGTH_SCALE = 1.0  # Vitesse de parole (1.0 = normal, 0.8 = rapide)

# Chemins (A adapter si besoin)
LOGS_DIR = "lightning_logs"
HIFIGAN_CONFIG = './checkpts/config.json'      
HIFIGAN_CHECKPT = './checkpts/generator_v1'

In [28]:
# 3. Fonctions Utilitaires (Copiées de vos scripts)

def get_latest_checkpoint(logs_dir="lightning_logs"):
    """Trouve le dernier checkpoint .ckpt généré par l'entraînement."""
    files = glob.glob(f"{logs_dir}/**/*.ckpt", recursive=True)
    if not files:
        raise FileNotFoundError(f"Aucun checkpoint trouvé dans {logs_dir}")
    latest_file = max(files, key=os.path.getmtime)
    print(f"📂 Checkpoint Matcha trouvé : {latest_file}")
    return latest_file

def load_hifigan(checkpoint_path, config_path, device):
    """Charge le modèle HiFi-GAN."""
    if not os.path.exists(config_path) or not os.path.exists(checkpoint_path):
        raise FileNotFoundError("Fichiers HiFi-GAN manquants !")

    with open(config_path) as f:
        data = f.read()
    json_config = json.loads(data)
    h = AttrDict(json_config)
    
    hifigan = HiFiGAN(h).to(device)
    state_dict = torch.load(checkpoint_path, map_location=device)
    
    if 'generator' in state_dict:
        hifigan.load_state_dict(state_dict['generator'])
    else:
        hifigan.load_state_dict(state_dict)
        
    hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan

In [29]:
# 4. Chargement des Modèles

# 1. Matcha TTS
ckpt_path = get_latest_checkpoint(LOGS_DIR)
matcha_model = MatchaTTS.load_from_checkpoint(ckpt_path, map_location=DEVICE)
matcha_model.eval()
matcha_model.to(DEVICE)
print("✅ Matcha-TTS chargé !")

# 2. HiFi-GAN
print("Chargement de HiFi-GAN...")
    
# 1. On lit la configuration
with open(HIFIGAN_CONFIG) as f:
    data = f.read()
json_config = json.loads(data)
h = AttrDict(json_config) # Utilisation de la classe AttrDict de env.py
    
# 2. On initialise le générateur
vocoder = HiFiGAN(h).to(DEVICE)
    
# 3. On charge les poids
# Note: Parfois le checkpoint contient tout un dictionnaire, parfois juste les poids.
# Le code officiel fait state_dict['generator']
state_dict_g = torch.load(HIFIGAN_CHECKPT, map_location=DEVICE)
vocoder.load_state_dict(state_dict_g['generator'])
    
vocoder.eval()
vocoder.remove_weight_norm() # Nettoyage pour l'inférence
print("HiFi-GAN chargé !")

📂 Checkpoint Matcha trouvé : lightning_logs\version_11\checkpoints\last.ckpt
✅ Matcha-TTS chargé !
Chargement de HiFi-GAN...
Removing weight norm...
HiFi-GAN chargé !


In [30]:
# 4. Paramètres de Génération

TEXTE = "Hello, I am your Matcha Text to Speech model, running inside a notebook!"
STEPS = 50           # Nombre d'étapes de diffusion (10-50)
TEMPERATURE = 0.667  # Variation (0.667 est standard)
LENGTH_SCALE = 1.0   # Vitesse (1.0 = normal, <1.0 = rapide)

print(f"Texte à synthétiser : '{TEXTE}'")

Texte à synthétiser : 'Hello, I am your Matcha Text to Speech model, running inside a notebook!'


In [31]:
# Préparation du texte
sequence = text_to_sequence(TEXTE, ["english_cleaners"])
x = torch.tensor([sequence], dtype=torch.long, device=DEVICE)
x_lengths = torch.tensor([len(sequence)], dtype=torch.long, device=DEVICE)

print("⏳ Génération du spectrogramme Mel...")
with torch.no_grad():
    # Utilisation directe de la méthode synthesise du modèle
    # Elle gère tout : encodeur, alignement, décodeur (CFM)
    output = matcha_model.synthesise(
        x=x, 
        x_lengths=x_lengths, 
        n_timesteps=STEPS,
        temperature=TEMPERATURE,
        length_scale=LENGTH_SCALE
    )
    
# Récupération du Mel
# output['mel'] est déjà dénormalisé par le modèle si les stats sont présentes
mel = output['mel']
    
print("✅ Spectrogramme généré !")
    
# Affichage du spectrogramme
mel_cpu = mel.squeeze().cpu().numpy()
plt.figure(figsize=(10, 4))
plt.imshow(mel_cpu, origin='lower', aspect='auto', cmap='inferno')
plt.colorbar(label='dB')
plt.title("Spectrogramme Mel")
plt.show()

⏳ Génération du spectrogramme Mel...
✅ Spectrogramme généré !


C:\Users\mathi\AppData\Local\Temp\ipykernel_35636\81470103.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [25]:
# 6. Vocoder (Mel -> Audio) et Écoute

if hifigan_model is not None and 'mel' in locals():
    print("⏳ Conversion Mel -> Audio avec HiFi-GAN...")
    with torch.no_grad():
        audio = hifigan_model(mel)
        audio = audio.squeeze().cpu()
        audio = audio.clamp(-1.0, 1.0) # Clipping de sécurité
        
    print("\n🎧 Résultat Audio :")
    display(Audio(audio, rate=22050))
    
    # Sauvegarde optionnelle
    save_path = "notebook_output.wav"
    torchaudio.save(save_path, audio.unsqueeze(0), 22050)
    print(f"Audio sauvegardé sous : {save_path}")
    
elif 'mel' in locals():
    print("⚠️ HiFi-GAN non chargé. Utilisation de Griffin-Lim (Qualité réduite)...")
    inv_mel_scale = torchaudio.transforms.InverseMelScale(
        n_stft=513, n_mels=80, sample_rate=22050, f_min=0.0, f_max=8000.0, 
        norm='slaney', mel_scale='slaney'
    ).to(DEVICE)
    griffin_lim = torchaudio.transforms.GriffinLim(n_fft=1024, n_iter=32).to(DEVICE)
    
    if mel.min() < 0: mel = torch.exp(mel) # Si Log-Mel
    waveform = griffin_lim(inv_mel_scale(mel))
    display(Audio(waveform.cpu(), rate=22050))

⚠️ HiFi-GAN non chargé. Utilisation de Griffin-Lim (Qualité réduite)...
